# Parte 2

## Árboles de Decisión

### Exploración, Preprocesamiento y Transformación de Datos

#### Imports

In [1]:
import numpy as np 
import pandas as pd
import re 

import matplotlib.pyplot as plt
import seaborn as sns

import os

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem import SnowballStemmer

import eli5

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from string import punctuation

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import joblib

#### Descripción de los Datos

#### Variables Faltantes

#### Correlaciones con la Cancelación

#### Limpieza del Dataset

## Generación y Evaluación de Modelos

### Árbol de Decisión

#### Entrenamiento

#### Gráfico del Modelo

#### Performance Review

### Random Forest

#### Entrenamiento

#### Gráfico del Modelo

#### Performance Review